In [39]:
import numpy as np
import bokeh.plotting
import bokeh.models
import bokeh.layouts
from bokeh import io

import pandas as pd

import discretize

from SimPEG import (
    simulation,
    maps,
    data_misfit,
    directives,
    optimization,
    regularization,
    inverse_problem,
    inversion,
)

bokeh.io.output_notebook()

Loading BokehJS ...

In [4]:
n_cells = 1000

mesh = discretize.TensorMesh([n_cells])

In [5]:
mesh

TensorMesh: 1,000 cells

                      MESH EXTENT             CELL WIDTH      FACTOR
  dir    nC        min           max         min       max      max
  ---   ---  ---------------------------  ------------------  ------
   x   1000          0.00          1.00      0.00      0.00    1.00

In [6]:
# build a model 

def gaussian(x, amplitude, mean, std):
    """Define a Gaussian Function"""
    return amplitude * np.exp(-((x-mean)/std)**2 / 2) 

def boxcar(x, amplitude, xlim): 
    """Define a Boxcar Function"""
    boxcar = np.zeros_like(x)  # initialize an array of all zeros
    boxcar_inds = (x >= xlim.min()) & (x <= xlim.max())  # find the indices of the boxcar
    boxcar[boxcar_inds] = amplitude
    return boxcar

In [7]:
def build_model(
    x, 
    amplitude_gaussian=1, mean_gaussian=0.7, std_gaussian=6e-2,
    amplitude_boxcar=1, xlim_boxcar=np.r_[0.2, 0.35]
):
    return (
        gaussian(x, amplitude_gaussian, mean_gaussian, std_gaussian) + 
        boxcar(x, amplitude_boxcar, xlim_boxcar)
    )

In [21]:
xc = mesh.gridCC
m = build_model(xc)

fig = bokeh.plotting.figure(title="true model", width=500, height=250)
fig.line(xc, m, line_width=2)
fig.xaxis.axis_label="x"

bokeh.io.show(fig)

In [36]:
def kernel_functions(x, j, p, q): 
    return np.exp(-p*j*x) * np.cos(2*np.pi*q*j*x) 

def generate_kernels(x, n_data=20, j_min=0, j_max=None, p=0.01, q=0.15):
    n_nodes = len(x)
    Gn = np.zeros((n_nodes, n_data))

    if j_max is None: 
        j_max = n_data
    j_values = np.linspace(j_min, j_max, n_data)
    for i, j in enumerate(j_values):
        Gn[:, i] = kernel_functions(xn, j, p, q)
    return Gn

In [48]:
xn = mesh.gridN
Gn = generate_kernels(xn)

fig = bokeh.plotting.figure(title="kernels", width=500, height=250)
# fig.line(xn, G, line_width=2)

fig.multi_line(xs=[xn]*20, ys=G.T.tolist())
# fig.
# fig.xaxis.axis_label="x"

bokeh.io.show(fig)


